# Import the necessary libraries

In [1]:
from glove import Corpus, Glove
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import datetime,time
from flashtext.keyword import KeywordProcessor
from nltk.stem import WordNetLemmatizer
from string import punctuation
import re
import pandas as pd
import numpy as np
import string
import multiprocessing
from nltk.corpus import stopwords
from flashtext.keyword import KeywordProcessor
from sklearn.model_selection import train_test_split
from sklearn import metrics

# libraries for dataset preparation, feature engineering, model training 
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
#from utils.language_models import WordVectorEmbedding
from gensim.test.utils import datapath, get_tmpfile
#import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.utils import to_categorical
from sklearn import preprocessing

C:\Program Files\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
C:\Program Files\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


# Import spooky author data from csv

In [2]:
df=pd.read_csv('train.csv',encoding='utf-8')
df.head()
sen=df['text']

# Preprocess data

In [3]:
# Define a function for removing regex
def regex_filtering(text):
        if text:
            #removing all email metadata fix it for email terms only
            text=re.sub(r"^(sender|to|copy|from|sent|subject|date|cc|e|von|datum|an|importance|bcc):.*$"," ",text,flags=re.M)
            #removing all mail ids
            text=re.sub(r"\S*@\S*\s?"," ",text)
            #removing all links
            text=re.sub(r"(((https?|ftp|file):\/\/)|www\\.)\\S+", ' ', text, flags=re.MULTILINE)
            text=re.sub(r"\w*\.\w{1,4}", '', text, flags=re.MULTILINE)
            #removing all non word character
            text=re.sub(r"([^a-zA-Z0-9\\u00C0-\\u00FF@]|[Ã£Ã¢])+",' ',text)
            #removing words with numbers 
            text=re.sub(r'\w*\d\w*', ' ', text)
            #removing single characters
            text=re.sub(r'\b\S{1}\s+',' ',text)
            #removing words with repeating characters
            text=re.sub(r'\b(\w)\1{1,}\s+',' ',text)
            #removing punkt
            text = text.translate(str.maketrans('','',string.punctuation))
            #removing extra whitespace
            text=re.sub(r"\s\s+",' ',text)
            #removing repeating words
            text=re.sub(r"(\w+\s+)\1{1,}",' ',text)
            #removing whitespaces
            text=text.strip()
            return text

In [4]:
#Tokenize Terms and remove stopwords
def tokenize_term(x):
        predefined_stopwords='horror perfectly'
        english_stopwords=stopwords.words("english")
        stopwords_list=(list(predefined_stopwords.split(' '))+english_stopwords)         
        keyword_processor_stopwords = KeywordProcessor()
        for each in stopwords_list:
            keyword_processor_stopwords.add_keyword(each,' ')   
        sentence=keyword_processor_stopwords.replace_keywords(x)
        return sentence.strip()

In [5]:
#Combined function
def preprocess(text):
    return regex_filtering(tokenize_term(text))

In [6]:
#Apply function preprocess and save result as new column
df['text_preprocessed']=df['text'].apply(preprocess)
texts=df['text_preprocessed']

In [7]:
df.head()

,id,text,author,text_preprocessed
0,id26305,"This process, however, afforded me no means of...",EAP,process however afforded means ascertaining di...
1,id17569,It never once occurred to me that the fumbling...,HPL,never occurred fumbling might mere mistake
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,left hand gold snuff box capered hill cutting ...
3,id27763,How lovely is spring As we looked from Windsor...,MWS,lovely spring looked Windsor Terrace sixteen f...
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,Finding nothing else even gold Superintendent ...


In [8]:
#Lemmatize different forms of words
def lemmatize_doc(dataframe):
    wordnet_lemmatizer = WordNetLemmatizer() 
    lemmatized = [[wordnet_lemmatizer.lemmatize(word,pos='v') for word in word_tokenize(s)]
                  for s in dataframe['text_preprocessed']]
    return lemmatized
lemma_data=lemmatize_doc(df)
df['text_lemmatized']=[" ".join(i) for i in lemma_data]
df.head()

,id,text,author,text_preprocessed,text_lemmatized
0,id26305,"This process, however, afforded me no means of...",EAP,process however afforded means ascertaining di...,process however afford mean ascertain dimensio...
1,id17569,It never once occurred to me that the fumbling...,HPL,never occurred fumbling might mere mistake,never occur fumble might mere mistake
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,left hand gold snuff box capered hill cutting ...,leave hand gold snuff box caper hill cut manne...
3,id27763,How lovely is spring As we looked from Windsor...,MWS,lovely spring looked Windsor Terrace sixteen f...,lovely spring look Windsor Terrace sixteen fer...
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,Finding nothing else even gold Superintendent ...,Finding nothing else even gold Superintendent ...


# Train model

## Defining model path

In [61]:
###Saving models with timestamp
import datetime,time

path="model/"
epochs=50
st = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d')+"_"
model_file_path =  path + "100D_spooky_author_glove_embedding_model_"+str(st) + str(epochs) + "_epoch.model"
glove_vectors_path = path + "100D_spooky_author_glove_embedding_model_" + str(st)+ str(epochs) + "_epoch.txt"
#glove_to_w2v_path = path + "spooky_author_glove_embedding_model_" +str(st)  + "_glove_to_word2vec.txt"

print (model_file_path)
print (glove_vectors_path)
#print(glove_to_w2v_path)

model/100D_spooky_author_glove_embedding_model_2019-03-02_50_epoch.model
model/100D_spooky_author_glove_embedding_model_2019-03-02_50_epoch.txt


## Set hyper-parameters for model training

In [62]:
#### Parameters for glove model
window = 5         ##### Contect window of every sentence it has to take
no_components=100  ##### No.of Dimensions or size of vectors
#no_components_100=100  ##### No.of Dimensions or size of vectors
learning_rate=0.05 ##### Learning rate or alpha
no_threads=84      ##### Multiprocessing ( specify the cores)
epochs=50

## Train glove embedding and save it

In [ ]:
# creating a corpus object
corpus = Corpus() 
#training the corpus to generate the co occurence matrix which is used in GloVe
corpus.fit(lemma_data, window=window)

#creating a Glove object which will use the matrix created in the above lines to create embeddings
#We can set the learning rate as it uses Gradient Descent and number of components
glove = Glove(no_components=no_components, learning_rate=learning_rate)
 
glove.fit(corpus.matrix, epochs=epochs, no_threads=no_threads, verbose=False)
glove.add_dictionary(corpus.dictionary)

In [63]:
glove.save(model_file_path)
print('Glove embedding model trained successfully. \nModel saved in the following path : {}'.format(model_file_path))

Glove embedding model trained successfully. 
Model saved in the following path : model/100D_spooky_author_glove_embedding_model_2019-03-02_50_epoch.model


## Save model in vector format

In [64]:
# Save model in vector format
word_vectors = glove.word_vectors

# Writing a vector file 
#Sample word : word1 0.2178995609092781 -0.3523739278544217 -0.14607002613618192 -0.0019162911041566608 

words = list(glove.dictionary)
word_vectors_for_words  = word_vectors.tolist()
glove_vectors = [m + " " + str(" ".join(str(item) for item in n)) for m,n in zip(words,word_vectors_for_words)] 

In [66]:
with open(glove_vectors_path,"w+") as fp:
    for line in glove_vectors:
        fp.write(str(line))
        fp.write("\n")
fp.close()

In [67]:
df.head()

,id,text,author,text_preprocessed,text_lemmatized
0,id26305,"This process, however, afforded me no means of...",EAP,process however afforded means ascertaining di...,process however afford mean ascertain dimensio...
1,id17569,It never once occurred to me that the fumbling...,HPL,never occurred fumbling might mere mistake,never occur fumble might mere mistake
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,left hand gold snuff box capered hill cutting ...,leave hand gold snuff box caper hill cut manne...
3,id27763,How lovely is spring As we looked from Windsor...,MWS,lovely spring looked Windsor Terrace sixteen f...,lovely spring look Windsor Terrace sixteen fer...
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,Finding nothing else even gold Superintendent ...,Finding nothing else even gold Superintendent ...


## Remove null rows present after preprocessing

In [68]:
filter_=df['text_lemmatized'].apply(lambda x:True if not x else False)

In [69]:
print("Before removing empty strings {}".format(filter_.sum()))
df=df[~filter_]
filter_=df['text_lemmatized'].apply(lambda x:True if not x else False)
print("After removing empty strings {}".format(filter_.sum()))

Before removing empty strings 0
After removing empty strings 0


In [70]:
df=df[~filter_]
df.to_pickle('df_authors_lemmatized_text.pkl')

## Find average vector of each sentence from the trained glove vector embeddings

In [78]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(df['text_lemmatized'])

sequences = tokenizer.texts_to_sequences(df['text_lemmatized'])
data = pad_sequences(sequences, maxlen=100)

In [79]:
print(data.shape)

(19573, 100)


### Build neural network with LSTM
#### Network Architechture
The network starts with an embedding layer. The layer lets the system expand each token to a more massive vector, allowing the network to represent a word in a meaningful way. The layer takes 20000 as the first argument, which is the size of our vocabulary, and 100 as the second input parameter, which is the dimension of the embeddings. The third parameter is the input_length of 50, which is the length of each comment sequence.

In [85]:
model_lstm = Sequential()
model_lstm.add(Embedding(20000, 100, input_length=100))
model_lstm.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_lstm.add(Dense(3, activation='softmax'))
model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [87]:
labels=df['author']
le = preprocessing.LabelEncoder()
labels=le.fit_transform(df['author'])
labels

array([0, 1, 0, ..., 0, 0, 1], dtype=int64)

In [89]:
from keras.utils import to_categorical
labels_cat = to_categorical(labels)
model_lstm.fit(data, np.array(labels_cat), validation_split=0.25, epochs=10)

Train on 14679 samples, validate on 4894 samples
Epoch 1/10
14679/14679 [==============================] - 73s 5ms/step - loss: 0.2094 - acc: 0.9278 - val_loss: 0.5588 - val_acc: 0.8081
Epoch 2/10
14679/14679 [==============================] - 69s 5ms/step - loss: 0.1093 - acc: 0.9611 - val_loss: 0.6540 - val_acc: 0.7949
Epoch 3/10
14679/14679 [==============================] - 78s 5ms/step - loss: 0.0758 - acc: 0.9725 - val_loss: 0.8168 - val_acc: 0.7983
Epoch 4/10
14679/14679 [==============================] - 77s 5ms/step - loss: 0.0624 - acc: 0.9769 - val_loss: 0.8540 - val_acc: 0.7940
Epoch 5/10
14679/14679 [==============================] - 72s 5ms/step - loss: 0.0429 - acc: 0.9855 - val_loss: 1.0004 - val_acc: 0.7850
Epoch 6/10
14679/14679 [==============================] - 68s 5ms/step - loss: 0.0412 - acc: 0.9853 - val_loss: 1.0108 - val_acc: 0.7906
Epoch 7/10
14679/14679 [==============================] - 67s 5ms/step - loss: 0.0312 - acc: 0.9897 - val_loss: 1.1257 - val_acc:

## Use Label-encoder to transform author class labels to machine understandable number inputs

In [100]:
#Using pretrained word vector
pretrained_vector_path='glove.6B.100d.txt'
embeddings_index = dict()
f = open(pretrained_vector_path,encoding='utf8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [101]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

##  same model architecture with a convolutional layer on top of the LSTM layer

In [104]:
import keras 
vocabulary_size=20000
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 100, input_length=100, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(100))
model_glove.add(Dense(3))
model_glove.add(Activation('softmax'))
model_glove.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [105]:
from keras.utils import to_categorical
labels_cat = to_categorical(labels)
model_glove.fit(data, np.array(labels_cat), validation_split=0.25, epochs = 10)

Train on 14679 samples, validate on 4894 samples
Epoch 1/10
14679/14679 [==============================] - 28s 2ms/step - loss: 0.8991 - acc: 0.5764 - val_loss: 0.8092 - val_acc: 0.6300
Epoch 2/10
14679/14679 [==============================] - 24s 2ms/step - loss: 0.7895 - acc: 0.6529 - val_loss: 0.7679 - val_acc: 0.6671
Epoch 3/10
14679/14679 [==============================] - 26s 2ms/step - loss: 0.7211 - acc: 0.6861 - val_loss: 0.7649 - val_acc: 0.6633
Epoch 4/10
14679/14679 [==============================] - 27s 2ms/step - loss: 0.6611 - acc: 0.7178 - val_loss: 0.7333 - val_acc: 0.6819
Epoch 5/10
14679/14679 [==============================] - 26s 2ms/step - loss: 0.6083 - acc: 0.7406 - val_loss: 0.7522 - val_acc: 0.6708
Epoch 6/10
14679/14679 [==============================] - 24s 2ms/step - loss: 0.5603 - acc: 0.7650 - val_loss: 0.7622 - val_acc: 0.6810
Epoch 7/10
14679/14679 [==============================] - 31s 2ms/step - loss: 0.5287 - acc: 0.7766 - val_loss: 0.7629 - val_acc: